<a href="https://colab.research.google.com/github/profteachkids/CHE5136_Fall2023/blob/main/ThreeTanks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import numpy as np
from scipy.integrate import solve_ivp
from plotly.subplots import make_subplots

In [28]:
Ac=1.
Ah = 0.3
g=9.81
qin0 = 0.1
qin = 1.2*qin0
tend=5.

h10 = ((qin0/Ah)**2)/2/g
h20 = h10
h30 = h10

In [29]:
def rhs(t, vec):
    h1, h2, h3 = vec
    q1, q2, q3 = Ah*np.sqrt(2*g)*np.sqrt(vec)
    dh1 = (qin - q1)/Ac
    dh2 = (q1 - q2)/Ac
    dh3 = (q2 - q3)/Ac
    return np.array([dh1, dh2, dh3])

In [35]:
res=solve_ivp(rhs, (0,tend), np.array([h10, h20, h30]), method='Radau', dense_output=True)

In [36]:
tplot = np.linspace(0,tend,200)

In [37]:
h1plot,h2plot,h3plot=res.sol(tplot)

In [38]:
fig=make_subplots()
fig.add_scatter(x=tplot,y=h1plot,mode='lines',name='h1')
fig.add_scatter(x=tplot,y=h2plot,mode='lines',name='h2')
fig.add_scatter(x=tplot,y=h3plot,mode='lines',name='h3')